# DCGAN

In [ ]:
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
tf.__version__

## 二次元人物头像-数据准备

In [ ]:
image_path = glob.glob(r"../input/anime-faces/data/*.png")

In [ ]:
len(image_path)

In [ ]:
def load_preprosess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_png(image,channels=3)
    image = tf.cast(image,tf.float32)
    image = (image/127.5) - 1
    return image

In [ ]:
image_ds = tf.data.Dataset.from_tensor_slices(image_path)

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [ ]:
image_ds = image_ds.map(load_preprosess_image,num_parallel_calls = AUTOTUNE)

In [ ]:
image_ds

In [ ]:
BATCH_SIZE = 64
image_count = len(image_path)

In [ ]:
image_ds = image_ds.shuffle(image_count).batch(BATCH_SIZE)
image_ds = image_ds.prefetch(AUTOTUNE)

In [ ]:
image_ds

In [ ]:
for item in image_ds:
    print(item.shape)
    print(type(item))
    break

## 生成器模型-反卷积

In [ ]:
def generator_model():
    
    model = tf.keras.Sequential()
    model.add(layers.Dense(8*8*256, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    model.add( layers.Reshape((8,8,256)) )  # 输出8*8*256
    # 反卷积
    model.add(layers.Conv2DTranspose(128,(5,5),strides=(1,1),padding='same',use_bias='false'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())      # 输出8*8*128
    
    model.add(layers.Conv2DTranspose(64,(5,5),strides=(2,2),padding='same',use_bias='false'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())      # 输出16*16*64
    
    model.add(layers.Conv2DTranspose(32,(5,5),strides=(2,2),padding='same',use_bias='false'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())      # 输出32*32*32
    
    model.add(layers.Conv2DTranspose(3,(5,5),
                                     strides=(2,2),
                                     padding='same',
                                     use_bias=False,
                                     activation='tanh'))  # 输出64*64*3
    
    
    return model

In [ ]:
# 测试生成器
generator = generator_model()
noise = tf.random.normal([1,100])
print(noise)
generated_image = generator(noise,training = False)
plt.imshow((generated_image[0,:,:,:3] + 1)/2)

## 判别器模型-卷积

In [ ]:
def discriminator_model():
    
    # 第一层 不用BatchNormalization
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(32,
                            (5,5),
                            strides = (2,2),
                            padding='same',
                            input_shape=(64,64,3))) # 输入为64*64*3
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3)) # 32*32*32
    
    
    
    model.add(layers.Conv2D(64,
                            (5,5),
                            strides = (2,2),
                            padding='same')) 
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3)) # 16*16*64
    
    
    
    model.add(layers.Conv2D(128,
                            (5,5),
                            strides = (2,2),
                            padding='same')) 
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3)) # 8*8*128
    
    model.add(layers.Conv2D(256,
                            (5,5),
                            strides = (2,2),
                            padding='same')) 
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())   # 4*4*256
    
    model.add(layers.GlobalAveragePooling2D())
    # Global Average Pooling(简称GAP，全局池化层)技术最早提出是在这篇论文（第3.2节）中，被认为是可以替代全连接层的一种新技术
    
    model.add(layers.Dense(1024))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Dense(1)) 
    
    return model
    

In [ ]:
# 测试一下判别器
discriminator = discriminator_model()
decision = discriminator(generated_image)
print(decision)

## 编写loss函数，定义优化器

In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True) 
# 因为判别器最后没有使用激活 所以我们添加from_logits=True

In [ ]:
# 求判别器损失的函数
def discriminator_loss(real_out,fake_out):
    real_loss = cross_entropy(tf.ones_like(real_out),real_out) # 真实图片的输出与1比较
    fake_loss = cross_entropy(tf.zeros_like(fake_out),fake_out) # 生成图片的输出与0比较
    return real_loss + fake_loss

In [ ]:
# 求生成器损失的函数
def generator_loss(fake_out):
    fake_loss = cross_entropy(tf.ones_like(fake_out),fake_out)
    return fake_loss

In [ ]:
# 优化器
generator_opt = keras.optimizers.Adam(1e-5)
discriminator_opt = keras.optimizers.Adam(1e-5)

In [ ]:
epoch_loss_avg_gen = tf.keras.metrics.Mean('g_loss')
epoch_loss_avg_disc = tf.keras.metrics.Mean('d_loss')
g_loss_results = []
d_loss_results = []

In [ ]:
EPOCHS = 300
noise_dim = 100 # 随机数的维度

num_example_to_generate = 6 # 用于绘图过程中生成图片的数量

seed = tf.random.normal([num_example_to_generate,noise_dim]) # 生成6个长度为100的随机向量

## 定义训练批次函数

In [ ]:
generator = generator_model() # 获取生成器模型
discriminator = discriminator_model() # 获取判别器模型

In [ ]:
# 接收一个批次的图片，对其进行训练
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE,noise_dim]) # 生成BTATH_SIZE个长度为100的随机向量
    # images 是真实图片的输入
    # noise 是噪声输入
    
    with tf.GradientTape() as gen_tape,tf.GradientTape() as disc_tape:
        real_out = discriminator(images,training = True)
        gen_image = generator(noise,training = True)
        fake_out = discriminator(gen_image,training = True)
        # 计算损失
        gen_loss = generator_loss(fake_out)
        disc_loss = discriminator_loss(real_out,fake_out)

        
    # 自动计算机损失函数关于自变量(模型参数)的梯度    
    gradient_gen = gen_tape.gradient(gen_loss,generator.trainable_variables)
    gradient_disc = disc_tape.gradient(disc_loss,discriminator.trainable_variables)
    # 根据梯度更新参数
    generator_opt.apply_gradients(zip(gradient_gen,generator.trainable_variables))
    discriminator_opt.apply_gradients(zip(gradient_disc,discriminator.trainable_variables))
    
    return gen_loss, disc_loss

## 训练与可视化

In [ ]:
# 画图函数
def generate_plot_image(gen_model,test_noise):
    pre_image = gen_model(test_noise,training = False)
    print(pre_image.shape)
    fig = plt.figure(figsize=(16,3)) # figsize:指定figure的宽和高，单位为英寸
    for i in range(pre_image.shape[0]):   # pre_image的shape的第一个维度就是个数，这里是6
        plt.subplot(1,6,i+1) # 几行几列的 第i+1个图片（从1开始）
        plt.imshow((pre_image[i,:,:,:] + 1)/2) # 加1除2: 将生成的-1～1的图片弄到0-1之间, gray设置灰色颜色
        plt.axis('off') # 不要坐标
    plt.show()

In [ ]:
# 训练(主方法)
def train(dataset,epochs):
    for epoch in range(1,epochs+1): # 总过训练epochs次
        print("\n epoch:",epoch)
        for image_batch in dataset: # 从数据集中遍历所有batch
            gen_loss, disc_loss = train_step(image_batch)# 训练一个batch
            
            epoch_loss_avg_gen(gen_loss)
            epoch_loss_avg_disc(disc_loss)
            
            print(".",end="")
        g_loss_results.append(epoch_loss_avg_gen.result())
        d_loss_results.append(epoch_loss_avg_disc.result())
        
        epoch_loss_avg_gen.reset_states()
        epoch_loss_avg_disc.reset_states()
        if epoch%5 == 0:
            generate_plot_image(generator,seed) # 绘图，用随机数seed在生成器
    generate_plot_image(generator,seed) # 绘图，用随机数seed在生成器

## 开始训练：

In [ ]:
train(image_ds,EPOCHS)

绘制损失折线图

In [ ]:
plt.plot(range(1, len(g_loss_results)+1), g_loss_results, label='g_loss')
plt.plot(range(1, len(d_loss_results)+1), d_loss_results, label='d_loss')
plt.legend()

## 使用生成器

In [ ]:
seed2 = tf.random.normal([30,noise_dim]) # 生成30个长度为100的随机向量
a = seed2.numpy()
pre = generator.predict(a)
print(pre[0].shape) # 查看输出的第一个图片的shape为(64, 64, 3)
plt.imshow((pre[0] + 1)/2, cmap = 'gray')